In [1]:
import praw
import glob
import os
from praw.models import MoreComments
import pandas as pd
from requests import Session
import datetime as dt
from datetime import date, timedelta
from psaw import PushshiftAPI
import emoji
import re

In [20]:
# Defining Functions

def get_date(created):
    return dt.datetime.fromtimestamp(created)

#___________________________________________________________________________________________________________
def redditing(start_date, end_date):
    try:
        # define connection variables
        reddit = praw.Reddit(
            client_id="ZjPaBT8ar8gCw7fwxI3d-Q",
            client_secret="HkSP6Qmf5cC5Pryh1e8JVasGiWo5Xg",
            password="1440falcon",  # pass Session"
            user_agent="API",
            username="alexcroitoru")


        api = PushshiftAPI()

        # define dates for query
        # start_time = int(dt.datetime(2020, month, day_start).timestamp())
        # end_time = int(dt.datetime(2020, month, day_end).timestamp())

        # begin query
        query = api.search_submissions(after=start_date, 
                                       before=end_date, 
                                       subreddit = "wallstreetbets")

        # create query and append submissions (threads)
        submissions = list()
        for element in query:
            submissions.append(element.d_)

        # convert list to dataframe
        reddits = pd.DataFrame(submissions)

        # filter on Daily Discussion Threads
        searchfor = ['Daily Popular Tickers Thread', 
                     'What Are Your Moves Tomorrow', 
                     'Daily Discussion Thread', 
                     'Weekend Discussion Thread']

       #reddit_api = reddits[reddits['title'].str.contains('|'.join(searchfor))]
        reddit_api = reddits[reddits['title'].str.contains("Daily Discussion Thread", regex=False)]

        reddit_id = reddit_api.iloc[0]['id']

        # pull id from DDT
        submission = reddit.submission(id = f'{reddit_id}')

        comments = list()
        postlist = []
        # Extract all comments
        submission.comments.replace_more(limit=0)

        #for comment in submission.comments.list():
            #comments.append(comment.body.d_)
            #reddit_comments = pd.DataFrame(comments)
            #return(reddit_comments)
            #return(comment.body)
        # Create DF for comments and add Column Names    
        for comment in submission.comments: 
                post = {}
                post['Submission'] = str(comment.submission)
                post['Author'] = str(comment.author)
                post['Comment'] = comment.body.encode('ascii', 'ignore').decode('ascii')
                post['Score'] = int(comment.score)
                post['Time Created'] = int(comment.created_utc)
                postlist.append(post)

                #print(postlist)

        Postdf = pd.DataFrame(postlist)
        return(Postdf)
    except Exception as E:
        print(E, starting_day)
        pass
#___________________________________________________________________________________________________________

In [21]:
# Define Start Date and End Date / Weekends are not ran and will throw a warning while running
first = 5
second = first+2

start_date = dt.date(2021, 8, 15)
delta = dt.timedelta(days=1)
window = start_date + dt.timedelta(days=2)
end_date = dt.date(2021, 8, 18)

starting_day = start_date.strftime('%d')[0:2]
ending_day = end_date.strftime('%d')[0:2]

month = start_date.strftime('%m')[1]

append_data = []

while start_date <= end_date:
    print(start_date)
    Postdf = pd.DataFrame(redditing(start_date=start_date, end_date=window))
    append_data.append(Postdf)
    start_date += delta
    window+=delta
    
appended_data = pd.concat(append_data)
_timestamp = appended_data["Time Created"].apply(get_date)
appended_data = appended_data.assign(timestamp = _timestamp)
appended_data = appended_data.drop(columns=["Time Created"])

excel_name = f'{month}_{starting_day}_{ending_day}_comments'
appended_data.to_csv(f'{excel_name}.csv')

2021-08-15
2021-08-16
2021-08-17
2021-08-18


In [17]:
os.chdir("../comment_folder")

In [18]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*comments.{}'.format(extension))]
all_filenames

['6_15_25_comments.csv',
 '6_25_05_comments.csv',
 '7_05_15_comments.csv',
 '7_15_25_comments.csv',
 '7_25_01_comments.csv',
 '8_2_17_comments.csv']

In [19]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
data['tokenized_sents'] = data.apply(lambda column: nltk.word_tokenize(column['Comment']), axis=1)
data